In [1]:
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request
import re
import time

In [ ]:
'''
setup - load entire page, pressing 'show more' button
'''

url = 'https://www.cardmarket.com/en/Magic/Products/Singles/Modern-Horizons/Snow-Covered-Island'
driver = webdriver.Chrome("/home/mig/Desktop/fcul 2019-2020/VD/project_mtg/selenium_drivers/chromedriver")
driver.get(url)

try:
    while True:
        driver.find_element_by_xpath(r"//span[contains(text(),'Show more results')]").click()
        time.sleep(2)
except ElementNotInteractableException as e:
    #now we have all the data, SCRAPE IT!!!
    pass
html = driver.page_source.encode('utf-8')

In [86]:
'''
setup - get the html to scrape
'''
'''
    setup without selenium:
'''  
'''
url = 'https://www.cardmarket.com/en/Magic/Products/Singles/Modern-Horizons/Snow-Covered-Island'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source, 'lxml')
table_tag = soup.find_all('div', class_="table-body")
table = table_tag[0]
info_tag = soup.find_all('div', class_='tab-container d-flex flex-column h-100')
info = info_tag[0]
'''

'''
    setup with selenium:
'''
soup = BeautifulSoup(html, 'lxml')
table_tag = soup.find_all('div', class_="table-body")
table = table_tag[0]
info_tag = soup.find_all('div', class_='tab-container d-flex flex-column h-100')
info = info_tag[0]
'''

'\nsoup = BeautifulSoup(html, \'lxml\')\ntable_tag = soup.find_all(\'div\', class_="table-body")\ntable = table_tag[0]\ninfo_tag = soup.find_all(\'div\', class_=\'tab-container d-flex flex-column h-100\')\ninfo = info_tag[0]\n'

In [87]:
'''
available_items
from
price_trend
30_days_average_price
7_days_average_price
1_day_average_price
'''
'''
tag = info.find_all('dl', class_='labeled row no-gutters mx-auto')
regex_available_items = r'Available\sitems</dt><dd\sclass="col-6\scol-xl-7">\d+<'
regex_from = r'From</dt><dd\sclass="col-6\scol-xl-7">\d+,\d+\s€<'
regex_price_trend = r'Price\sTrend</dt><dd\sclass="col-6\scol-xl-7"><span>\d+,\d+\s€<'
regex_30_days_average_price = r'30-days\saverage\sprice</dt><dd\sclass="col-6\scol-xl-7"><span>\d+,\d+\s€<'
regex_7_days_average_price = r'7-days\saverage\sprice</dt><dd\sclass="col-6\scol-xl-7"><span>\d+,\d+\s€<'
regex_1_day_average_price = r'1-day\saverage\sprice</dt><dd\sclass="col-6\scol-xl-7"><span>\d+,\d+\s€<'
match1 = re.findall(regex_available_items, str(tag))
match2 = re.findall(regex_from, str(tag))
match3 = re.findall(regex_price_trend, str(tag))
match4 = re.findall(regex_30_days_average_price, str(tag))
match5 = re.findall(regex_7_days_average_price, str(tag))
match6 = re.findall(regex_1_day_average_price, str(tag))
display(match1[0][47:-1])
display(match2[0][36:-3])
display(match3[0][49:-3])
display(match4[0][59:-3])
display(match5[0][58:-3])
display(match6[0][57:-3])

_available_items = match1[0][47:-1]
_from = match2[0][36:-3]
_price_trend = match3[0][49:-3]
_30_days_average_price = match4[0][59:-3]
_7_days_average_price = match5[0][58:-3]
_1_day_average_price = match6[0][57:-3]
'''

'17855'

'0,09'

'0,38'

'0,38'

'0,39'

'0,38'

In [88]:
#get all rows in table
row_tags = table.find_all('div', class_='row no-gutters article-row')

In [93]:
def get_sales_available_items(row_tag):
    tag=row_tag.find_all('span', class_='badge badge-faded d-none d-sm-inline-flex has-content-centered mr-1 sell-count')[0]#.get_text().strip()
    regex = r'\d+\sSales\s|\s\d+\sAvailable\sitems'
    match = re.findall(regex, str(tag))
    sales = match[0][:-7]
    available_items = match[1][1:-16]
    return sales, available_items

def get_item_conditions_languages_is_playset(row_tag):
    tag=row_tag.find_all('div', class_='product-attributes col-auto col-md-12 col-xl-5')[0]#.get_text().strip()
    regex = r'showMsgBox\(this,\'.*?\'\)'
    match = re.findall(regex, str(tag))
    item_conditions = match[0][17:-2]
    item_languages = match[1][17:-2]
    item_is_playset = True if len(match) == 3 else False
    return item_conditions, item_languages, item_is_playset
    
'''
iterate through each row in the table
'''
seller_names = []
item_prices = []
item_amounts = [] 
seller_sales = []
seller_available_items = []
item_locations = []
item_conditions = []
item_languages = []
item_is_playsets = []
for row_tag in row_tags:
    seller_names.append(row_tag.find_all('span', class_='d-flex has-content-centered mr-1')[0].get_text().strip())
    
    item_prices.append(row_tag.find_all('span', class_='font-weight-bold color-primary small text-right text-nowrap')[0].get_text().strip()[:-2])
    
    item_amounts.append(row_tag.find_all('span', class_='item-count small text-right')[0].get_text().strip()[:])
    
    sales, available_items = get_sales_available_items(row_tag)
    seller_sales.append(sales)
    seller_available_items.append(available_items)
    
    item_locations.append(row_tag.find_all('span', class_='icon d-flex has-content-centered mr-1')[0]['title'][15:])
    
    item_condition, item_language, item_is_playset = get_item_conditions_languages_is_playset(row_tag)
    item_conditions.append(item_condition)
    item_languages.append(item_language)
    item_is_playsets.append(item_is_playset)

'''
change some datatypes
'''
item_prices = [float(x.replace(',','.')) for x in item_prices]
item_amounts = [int(x) for x in item_amounts]

'''
correct for playsets:
'''
for i in range(len(item_is_playsets)):
    if item_is_playsets[i] == True:
        item_prices[i] = item_prices[i] / 4
        item_amounts[i] = item_amounts[i] * 4
    
display('seller_names', len(seller_names), seller_names[:10], 
    'item_prices', len(item_prices), item_prices[:10], 
    'item_amounts', len(item_amounts), item_amounts[:10], 
    'item_locations', len(item_locations), item_locations[:10], 
    'item_conditions', len(item_conditions), item_conditions[:10], 
    'item_languages', len(item_languages), item_languages[:10], 
    'item_is_playsets' ,len(item_is_playsets), item_is_playsets[:])

old price 0.990000
new price 0.247500


'seller_names'

50

['toycastle',
 'talcoon85',
 'greggoss',
 'svilla92',
 'Ramix',
 'Soldatpacif',
 'xFL0x',
 'Gui-B',
 'arnaujuanco',
 'Alphannibal']

'item_prices'

...

50

['English',
 'English',
 'French',
 'Spanish',
 'English',
 'French',
 'German',
 'French',
 'English',
 'French']

'item_is_playsets'

50

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False]

In [ ]:
'''
put into pandas
'''
data_dict = {
    'timestamp' = [pd.Timestamp.now(tz='UTC') for i in range(len(seller_names))],
    'seller_names' = seller_names,
    'seller_names' = seller_names,
    'seller_names' = seller_names,
    'seller_names' = seller_names,
    'seller_names' = seller_names,
    'seller_names' = seller_names,
    'seller_names' = seller_names,
}
seller_names = []
seller_sales = []
seller_available_items = []
item_prices = []
item_amounts = [] 
item_locations = []
item_conditions = []
item_languages = []
item_is_playsets = []

In [94]:
pd.Timestamp.now(tz='UTC')

Timestamp('2019-10-07 18:30:47.811965+0000', tz='UTC')